In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Base directory containing the dataset
base_dir = 'C:/Users/pahanibihanga/Videos/rubber_leaves'

# List of all diseases (subdirectories), including the new 'non_rubber' class
diseases = [
    'birds_eye_spot',
    'colletotrinchum_leaf_spot',
    'corynespora_leaf',
    'phytopthora_leaf_fall',
    'powdery_mildew',
    'rubber',
    'non_rubber'
]

# ImageDataGenerator for data augmentation
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, horizontal_flip=True)
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load dataset from base directory
train_generator = train_datagen.flow_from_directory(
    base_dir, target_size=(224, 224), batch_size=32, class_mode='categorical', classes=diseases
)
valid_generator = valid_datagen.flow_from_directory(
    base_dir, target_size=(224, 224), batch_size=32, class_mode='categorical', classes=diseases
)
test_generator = test_datagen.flow_from_directory(
    base_dir, target_size=(224, 224), batch_size=32, class_mode='categorical', shuffle=False, classes=diseases
)

# Load MobileNet (pre-trained model)
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze initial layers

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(len(diseases), activation='softmax')(x)  # Output layer

# Create model
model = Model(inputs=base_model.input, outputs=x)

# Compile model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
history = model.fit(
    train_generator, epochs=10, validation_data=valid_generator, verbose=1
)

# Fine-tuning: Unfreeze some layers
base_model.trainable = True
for layer in base_model.layers[:100]:  # Keep first 100 layers frozen
    layer.trainable = False

# Recompile model with lower learning rate
model.compile(optimizer=Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train with early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
history_fine_tune = model.fit(
    train_generator, epochs=10, validation_data=valid_generator, verbose=1, callbacks=[early_stopping]
)

# Evaluate model
test_loss, test_acc = model.evaluate(test_generator, verbose=1)
print(f"Test accuracy: {test_acc:.2f}")

# Save model
model.save('rubber_tree_disease_model.h5')

# Load saved model
model = load_model('rubber_tree_disease_model.h5')

# Function to prepare a single image
def prepare_image(img_path, target_size=(224, 224)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0  # Normalize
    return img_array

# Function to predict class with confidence
def predict_image(model, img_path, threshold=0.6):
    processed_image = prepare_image(img_path)
    prediction = model.predict(processed_image)

    # Get the highest probability and class index
    max_prob = np.max(prediction)
    predicted_class_idx = np.argmax(prediction)

    # If confidence is low, classify as "Not a Rubber Leaf"
    if max_prob < threshold:
        return "Not a Rubber Leaf", max_prob

    return diseases[predicted_class_idx], max_prob

# Example: Test with an image
img_path = '1.jpg'  # Replace with actual image path
predicted_label, confidence = predict_image(model, img_path)
print(f"Predicted class for {img_path}: {predicted_label} (Confidence: {confidence:.2f})")

# Plot accuracy and loss
plt.figure(figsize=(12, 4))

# Accuracy plot
plt.subplot(1, 2, 1)
plt.plot(history_fine_tune.history['accuracy'], label='Train')
plt.plot(history_fine_tune.history['val_accuracy'], label='Validation')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Loss plot
plt.subplot(1, 2, 2)
plt.plot(history_fine_tune.history['loss'], label='Train')
plt.plot(history_fine_tune.history['val_loss'], label='Validation')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()


img_path = '19.JPG'  # Replace with actual image path
predicted_label, confidence = predict_image(model, img_path)
print(f"Predicted class for {img_path}: {predicted_label} (Confidence: {confidence:.2f})")


import tensorflow as tf

# Load the trained model
model = tf.keras.models.load_model('rubber_tree_disease_model.h5')

# Convert to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLite model
with open('rubber_tree_disease_model.tflite', 'wb') as f:
    f.write(tflite_model)

print("Model successfully converted to TFLite format!")

img_path = '17.jpg'  # Replace with actual image path
predicted_label, confidence = predict_image(model, img_path)
print(f"Predicted class for {img_path}: {predicted_label} (Confidence: {confidence:.2f})")

img_path = '8.jpeg'  # Replace with actual image path
predicted_label, confidence = predict_image(model, img_path)
print(f"Predicted class for {img_path}: {predicted_label} (Confidence: {confidence:.2f})")